In [82]:
import tensorflow as tf

tf.debugging.set_log_device_placement(True)
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

from tensorflow.python.client import device_lib
from tensorflow.keras.applications import VGG16

print(device_lib.list_local_devices())
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Flatten,BatchNormalization

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16590905566570605819
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2248620032
locality {
  bus_id: 1
  links {
  }
}
incarnation: 831039352747856756
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1660, pci bus id: 0000:26:00.0, compute capability: 7.5"
]


In [ ]:
# 사전 학습된 모델 불러오기
input_tensor = Input(shape=(224,224,3))
vgg_model = VGG16(weights='imagenet', include_top=False, input_tensor = input_tensor)
#vgg_model.trainable=False
vgg_model.summary()

Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/tas

In [24]:
vgg_model.trainable=False
#for layer in vgg_model.layers[:-4]:
#    layer.trainable=False
flatten_layer = Flatten()
bn= layers.BatchNormalization()
dense_layer_1 = Dense(128, activation='relu')
do=Dropout(.3)
prediction_layer  = Dense(4, activation='softmax')
        
model = Sequential([
    vgg_model,
    flatten_layer,
    dense_layer_1,
    do,
    prediction_layer
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               3211392   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 516       
Total params: 17,926,596
Trainable params: 3,211,908
Non-trainable params: 14,714,688
_________________________________________________________________


In [25]:
from tensorflow.keras.optimizers import Adam,Nadam
model.compile(loss='categorical_crossentropy',
                     optimizer=Nadam(0.0001),
                     metrics=['accuracy'])

In [26]:
train_dir ='D:\\빅데이터기반지능형서비스개발\\07기업요구사항기반의문제해결pjt\\tdata\\train'
test_dir = 'D:\\빅데이터기반지능형서비스개발\\07기업요구사항기반의문제해결pjt\\tdata\\validation'

train_image_generator = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,
                                          horizontal_flip=True,
                                          width_shift_range=.2,height_shift_range=.2,
                                          fill_mode='nearest')
val_image_generator = ImageDataGenerator(rescale=1./255)
### augmentation : ImageDataGenerator( rescale = 1.0/255,shear_range=0.2,zoom_range=0.2)

train_data_gen = train_image_generator.flow_from_directory(batch_size=32,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(224, 224),
                                                           class_mode='categorical')

val_data_gen = val_image_generator.flow_from_directory(batch_size=32,
                                                         directory=test_dir,
                                                         target_size=(224, 224),
                                                         class_mode='categorical')

Found 28873 images belonging to 4 classes.
Found 8248 images belonging to 4 classes.


In [41]:
from collections import Counter
counter = Counter(train_data_gen.classes)
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images/10 for class_id, num_images in counter.items()}                     
class_weights


{0: 3.101123595505618, 1: 0.1, 2: 0.1738947810563898, 3: 0.7340425531914894}

In [80]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc', 
    verbose=1,
    patience=2,
    mode='max',
    restore_best_weights=True)

NameError: name 'tf' is not defined

In [81]:
from tensorflow.keras.models import load_model
history = model.fit_generator(train_data_gen,
                                epochs=10,
                                validation_data=val_data_gen,
                              shuffle=True,callbacks=[early_stopping],
                               class_weight=class_weight)

model.save("logs/n_tran1.h5")

NameError: name 'model' is not defined